In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [3]:
env = gym.make("CartPole-v0")

In [4]:
print(env.action_space.sample())

0


In [5]:
 class ReplayMemory:
    #rotating buffer of size N
    def __init__(self,N,batch_size=500):
        self.memory = []
        self.size = N
        self.batch_size =batch_size
    
    def add(self,experience):
        self.memory.append(experience)
        if(len(self.memory) > self.size):
            del self.memory[0]
    
    def replay(self):
        
        # we want to keep some information about direction though 
        # so we will give 3 frames at a time
        
        buffer_size = len(self.memory)
        output = np.array(self.memory)
        
        if(buffer_size < self.batch_size):
            return []
        
        index = np.random.choice(np.arange(buffer_size),
                                size = self.batch_size,
                                replace = False)
        return output[index]

#Define what our experience looks like
#[state,action,reward,next_state]

In [6]:
number_of_actions = 2
# DQN network from this tutorial 
# https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288
# We will probably tweak this but our main interesting part is training this model
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        # 4 input image channel, 32 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv1d(5, 32, 3)
        #self.conv2 = nn.Conv2d(32, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, number_of_actions)

    def forward(self, x):
        
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
#cut off the infromation on the top and grayscale
def format_frame(frame):
    frame =np.mean(frame,axis=2)
    frame = frame[35:-15]
    return frame

def get_most_likely_action(action_confidences):
    #returns the index of the most likely action
    return np.argmax(action_confidences)

def to_onehot(index,size):
    #makes a onehot array of size size
    # with the index index 1 and all others 0
    onehot = torch.zeros(size)
    onehot[index] = 1
    return onehot

def format_frames(stacked_frames):
    #print(stacked_frames)
    # turn a list of numpy arrays into 
    # a torch tensor
    torch_frames = [torch.from_numpy(n).float() for n in stacked_frames]
    #now stack them into one big tensor
    output_tensor = torch.stack(torch_frames)
    output_tensor = torch.reshape(output_tensor,(1,len(stacked_frames),-1))
    return output_tensor

def get_best_action(model,stacked_frames):
    #get our best action from our learner
    #print("state given:",state)
    frames_tensor = format_frames(stacked_frames)
    
    action = model.forward(frames_tensor).detach().numpy()[0]
    #action = int(action)
    #print("State:",state,"Action:",action)
    action = get_most_likely_action(action)
    return int(action)

In [8]:

def learn_from_data(model,target,data,optim):
    for experience in data:
        #compute the loss from
        state,action,reward,next_state = experience
        #make the action match what the network outputs
        
        #belman equation
        state = format_frames(state)
        next_state = format_frames(next_state)
        
        chosen_action_based_on_reward = model.forward(state)
        
        expected_reward = target.forward(next_state).detach_()
        
        reward_tensor = torch.zeros(number_of_actions)
        reward_tensor[action] = reward
        
        #reward_tensor.float()
        #print(reward_tensor)
        
        expected_reward += reward_tensor
        
        loss = F.l1_loss(chosen_action_based_on_reward,expected_reward)
        #we want the chosen_action based on reward to match 
        #the reward of being in the next state and the reward given
        
        #print(reward)
        
        # must zero gradients before backprop
        # for pytorch
        optim.zero_grad()
        model.zero_grad()
        loss.backward()
        optim.step()
    return model

In [9]:
def select_action(exploration_rate,state,model):
    uniform_random_variable= random.uniform(0,1)
        
    if(uniform_random_variable > exploration_rate):
        
        #state = torch.tensor([[state]])
        #state=state.float()

        action = get_best_action(model,state)
        
    else:
        #other wise explore randomly
        action = env.action_space.sample()
    return action

In [10]:
def update_target(model,target):
    target.load_state_dict(model.state_dict())
    #print("model:",model.state_dict())
    #print("target:",target.state_dict())
    return target

In [11]:
# Run the training loop
# Currently we do not have a target network but we could do that
num_episodes = 1000000
training_episodes = 10
target_episodes = 15
# network_update_epochs = 100 #use this if using a target network
game_length = 10000

# exploration rate, we want our network to explore 
# sometimes but not all the time, to do this
# we use a decaying exploration rate
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.01
# this bit of code will determine the decay
# exploration_rate = min_exploration_rate + \
#        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

In [12]:
replay_memory = ReplayMemory(10000)
dqn = DQN()#96,96)
target = DQN()
optimizer = optim.SGD(dqn.parameters(),lr=.01)
print(dqn)

DQN(
  (conv1): Conv1d(5, 32, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=16, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)


In [13]:
target = update_target(dqn,target)

def play_game(dqn,replay_memory,env,bounding_reward = True,show=False):
    time.sleep(.1)
    frame= env.reset()
    #frame = format_frame(frame)
    
    episode_reward = 0
    tko_timer = 0
    game = []
    
    action_frequency = 5
    
    stacked_frames = []
    previous_frames = []
    stacked_reward = 0
    #stacked_frames.append(frame)
    
    action = env.action_space.sample()
    
    
    for step in range(game_length):
        #if(show==True):
            #env.render()
        #only make a new desision every 5 frames
        
        frame, reward, done, info = env.step(action)
        
        
        #frame = format_frame(frame)
        stacked_frames.append(frame)
        
        #env.render() we could render to show ourseleves
        if(step % action_frequency==0 and step != 0):
            action = select_action(exploration_rate,stacked_frames,dqn)
        
        
        
        
        if(reward < 0 and bounding_reward):
            reward = 0
        
        stacked_reward += reward
        
        
        
        if(step % action_frequency==0 and step > action_frequency):
            experience= [previous_frames,action,stacked_reward,stacked_frames]
            replay_memory.add(experience)
        
        if(step % action_frequency==0):
            previous_frames=stacked_frames
            stacked_frames = []
            stacked_reward = 0
            
        
        
        
        
         
        
        
        #tracking how far we are getting
        episode_reward +=reward
        
        if(done):
            # if we haven't gotten a positive reward 
            # in the last 20 steps 
            # or the game is over: stop
            break
    return episode_reward

def one_tenth_average(array):
    #take the average of every 10 elements
    output = []
    for i in range(0,len(array),10):
        output.append(sum(array[i:(i+10)])/10.0)
    return output
    
def show_progress(rewards):
    
    while(len(rewards) > 100):
        #take the average of the last 10
        rewards = one_tenth_average(rewards)
    
    
    clear_output(wait=True)
    plt.plot(rewards)
    plt.show()
    print("exploration_rate:",exploration_rate)

In [15]:
rewards = []
for episode in range(num_episodes):
    print("Episode:",episode)
    
    #the play_game will update replay_memory to include the game
    episode_reward = play_game(dqn,replay_memory,env,bounding_reward = True,show=(episode%20==0))
    
    
    #end of episode variable updating
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    rewards.append(episode_reward)
    
    show_progress(rewards)
    
    if(episode % training_episodes == 0 and episode != 0):
        print("Training on Experience")
        dqn = learn_from_data(dqn,target,replay_memory.replay(),optimizer)
    
    if(episode%target_episodes ==0):
        print("updating the target network")
        target = update_target(dqn,target)
    

KeyboardInterrupt: 